In [ ]:
pip install cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 70.5 MB/s eta 0:00:00


In [ ]:
!apt-get -qq install -y graphviz
!pip install pydot
import pydot


In [ ]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 126308 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.6.0-1ubuntu1.5_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.6.0-1ubuntu1.5) ...
Setting up libarchive-dev:amd64 (3.6.0-1ubuntu1.5) ...
Processing triggers for man-db (2.10.2-1) ...
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 8.0 MB/s eta 0:00:00
  Created wheel for libarchive: filename=libarchive-0.4.7-py3-none-any.whl size=31629 sha256=ca1169fbf8fdc6b6f37c2710d38000ffb39a57d84ff27609eb3cb8fdc0d86f08
  Stored in directory: /root/.cache/pip/wheels/32/98/bd/4893d6923dd027f455b250367d402bfd69a6f4416581df46db
Successfully built libarchive


In [ ]:
from google.colab import files
# This will open a file picker to upload from your computer
uploaded = files.upload()

Saving easy_sales_data_1000.csv to easy_sales_data_1000.csv


In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum as Fsum

# Step 1: Start Spark session
spark = SparkSession.builder.appName("RetailSalesAnalysis").getOrCreate()

# Step 2: Load CSV
df = spark.read.option("header", "true").option("inferSchema", "true").csv("easy_sales_data_1000.csv")

# Step 3: Clean nulls & standardize column names
df_clean = df.dropna()
new_columns = [c.strip().lower().replace(" ", "_") for c in df_clean.columns]
df_clean = df_clean.toDF(*new_columns)

# Step 4: Average sales by region
avg_sales = df_clean.groupBy("region").agg(avg("sales").alias("avg_sales"))
avg_sales.show()

# Step 5: Revenue share per category
category_totals = df_clean.groupBy("category").agg(Fsum("sales").alias("total_category_sales"))

# Join with original
df_joined = df_clean.join(category_totals, on="category", how="inner")

# Add revenue share column
df_final = df_joined.withColumn("category_revenue_share", col("sales") / col("total_category_sales"))

# Show final results
df_final.select("product_id", "category", "sales", "category_revenue_share").show(10)


+------+------------------+
|region|         avg_sales|
+------+------------------+
|  west|2599.0597244094465|
| north|2693.6103125000004|
|  east|2647.4929875518656|
| south| 2679.598112449797|
+------+------------------+

+----------+---------+-------+----------------------+
|product_id| category|  sales|category_revenue_share|
+----------+---------+-------+----------------------+
|   PRD0001|   drinks|3716.16|  0.007833035869248906|
|   PRD0002|   frozen|4719.67|  0.010834169061216058|
|   PRD0003|household|4605.15|  0.011165121654888189|
|   PRD0004|   drinks|4757.34|   0.01002766696326654|
|   PRD0005|    dairy|1601.22|  0.003180283284386013|
|   PRD0006|   frozen|1092.04|  0.002506816362502...|
|   PRD0007|household|1860.25|  0.004510150062105633|
|   PRD0008|    dairy|4527.88|  0.008993105930294239|
|   PRD0009|    dairy|4874.24|  0.009681033209731129|
|   PRD0010|    dairy|4366.96|  0.008673492644097839|
+----------+---------+-------+----------------------+
only showing top 10

In [18]:
#select()
df.select("product_id", "category", "sales").show(5)

+----------+---------+-------+
|product_id| category|  sales|
+----------+---------+-------+
|   PRD0001|   drinks|3716.16|
|   PRD0002|   frozen|4719.67|
|   PRD0003|household|4605.15|
|   PRD0004|   drinks|4757.34|
|   PRD0005|    dairy|1601.22|
+----------+---------+-------+
only showing top 5 rows



In [19]:
#withcolumn
df.withColumn("Discount", df.sales * 0.2).show(5)

+----------+------+-------+----------+---------+------+--------+----------+----------+------+-----------+-------+-----------------+
|product_id|weight|    fat|visibility| category| price|store_id|store_year|store_size|region| store_type|  sales|         Discount|
+----------+------+-------+----------+---------+------+--------+----------+----------+------+-----------+-------+-----------------+
|   PRD0001| 10.62|Low Fat|     0.026|   drinks|170.21|  STR103|      2012|      high| north|hypermarket|3716.16|          743.232|
|   PRD0002| 19.26|Regular|     0.024|   frozen| 79.71|  STR104|      2019|    medium| north|supermarket|4719.67|943.9340000000001|
|   PRD0003| 15.98|Regular|    0.0013|household|105.47|  STR103|      2002|    medium| north|supermarket|4605.15|           921.03|
|   PRD0004| 13.98|Regular|    0.0171|   drinks| 33.73|  STR105|      2019|      high|  west|hypermarket|4757.34|951.4680000000001|
|   PRD0005|  7.34|Regular|     0.019|    dairy| 81.18|  STR101|      2003| 

In [20]:
#Groupby.aggregate
df.groupBy("region").agg(avg("sales").alias("avg_sales")).show(5)

+------+------------------+
|region|         avg_sales|
+------+------------------+
|  west|2599.0597244094465|
| north|2693.6103125000004|
|  east|2647.4929875518656|
| south| 2679.598112449797|
+------+------------------+

